tutorial from https://medium.com/analytics-vidhya/training-an-object-detection-model-with-tensorflow-api-using-google-colab-4f9a688d5e8b

Degrade tf version to be able to use object_detection


In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


Check that GPU is enabled


In [2]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
   raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
print(tf.__version__)

SystemError: ignored

Move to training repository (check that google drive is installed) 

In [3]:
%cd '/content/drive/My Drive/training_demo/'

/content/drive/My Drive/training_demo



Clone the tf models repository





In [0]:
!git clone https://github.com/tensorflow/models.git


fatal: destination path 'models' already exists and is not an empty directory.


Install packages and dependencies

In [4]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
python-tk is already the newest version (2.7.17-1~18.04).
The following additional packages will be installed:
  python-bs4 python-chardet python-html5lib python-olefile
  python-pkg-resources python-six python-webencodings
Suggested packages:
  python-genshi python-lxml-dbg python-lxml-doc python-pil-doc python-pil-dbg
  python-setuptools
The following NEW packages will be installed:
  python-bs4 python-chardet python-html5lib python-lxml python-olefile
  python-pil python-pkg-resources python-six python-webencodings
0 upgraded, 9 newly installed, 0 to remove and 29 not upgraded.
Need to get 1,818 kB of archives.
After this operation, 7,685 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python-bs4 all 4.6.0-1 [67.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd

Compile the model definition

In [5]:
%cd /content/drive/My Drive/training_demo/models/research/
!protoc object_detection/protos/*.proto --python_out=.

/content/drive/My Drive/training_demo/models/research


Set the environment path

In [0]:
import os
os.environ['PYTHONPATH'] += ':/content/drive/My Drive/training_demo/models/research/:/content/drive/My Drive/training_demo/models/research/slim'

Setup object_detection

In [0]:
!python setup.py build
!python setup.py install

running build
running build_py
copying object_detection/protos/anchor_generator_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/argmax_matcher_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/bipartite_matcher_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/box_coder_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/box_predictor_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/calibration_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/eval_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/hyperparams_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/image_resizer_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/faster_rcnn_box_coder_pb2.py -> build/lib/object_detection/protos
copying object_detection/protos/input_reader_pb2.py -> build/lib/objec

Compute remaining time for the session

In [0]:
import time, psutil
Start = time.time()- psutil.boot_time()
Left= 12*3600 - Start
print('Time remaining for this session is: ', Left/3600)

Time remaining for this session is:  11.639220221108861


Test installation

In [0]:
%cd /content/drive/My Drive/training_demo/models/research/object_detection/builders/
!python model_builder_test.py

/content/drive/My Drive/training_demo/models/research/object_detection/builders
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Running tests under Python 3.6.9: /usr/bin/python3
[ RUN      ] ModelBuilderTest.test_create_experimental_model
[       OK ] ModelBuilderTest.test_create_experimental_model
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[       OK ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_with_matmul
[       OK ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_with_matmul
[ RUN  

partition data set into test and train (jpg+xml.files need to be in folder 'images')

In [0]:
%cd /content/drive/My Drive/training_demo/preprocessing

#10 percent test data
!python partition_dataset.py -x -i /content/drive/My\ Drive/training_demo/images/images_small -r 0.1



/content/drive/My Drive/training_demo/preprocessing


KeyboardInterrupt: ignored

Create csv-files 

In [0]:
%cd /content/drive/My Drive/training_demo/preprocessing
!python xml_to_csv.py -i /content/drive/My\ Drive/training_demo/images/images_small/train -o /content/drive/My\ Drive/training_demo/annotations/train_labels.csv
!python xml_to_csv.py -i /content/drive/My\ Drive/training_demo/images/images_small/test -o /content/drive/My\ Drive/training_demo/annotations/test_labels.csv


/content/drive/My Drive/training_demo/preprocessing
Successfully converted xml to csv.
Successfully converted xml to csv.


Generate TF-records

In [0]:
%cd /content/drive/My Drive/training_demo/preprocessing

#create train data:
!python generate_tfrecord.py --label=potato --csv_input=/content/drive/My\ Drive/training_demo/annotations/train_labels.csv --img_path=/content/drive/My\ Drive/training_demo/images/images_desk --output_path=/content/drive/My\ Drive/training_demo/annotations/train.record

# Create test data:
#!python generate_tfrecord.py --label=potato --csv_input=/content/drive/My\ Drive/training_demo/annotations/test_labels.csv --img_path=/content/drive/My\ Drive/training_demo/images/images_small/test --output_path=/content/drive/My\ Drive/training_demo/annotations/test.record


/content/drive/My Drive/training_demo/preprocessing


MessageError: ignored

Get pre-trained model

In [0]:
%cd /content/drive/My Drive/training_demo/pre-trained-model


!wget http://download.tensorflow.org/models/object_detection/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz
!tar -xvf ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz

/content/drive/My Drive/training_demo/pre-trained-model
--2020-05-08 20:57:50--  http://download.tensorflow.org/models/object_detection/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.20.128, 2607:f8b0:400e:c07::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.20.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51025348 (49M) [application/x-tar]
Saving to: ‘ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz’

ssdlite_mobilenet_v 100%[===================>]  48.66M  86.6MB/s    in 0.6s    

2020-05-08 20:57:51 (86.6 MB/s) - ‘ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz’ saved [51025348/51025348]

ssdlite_mobilenet_v2_coco_2018_05_09/checkpoint
ssdlite_mobilenet_v2_coco_2018_05_09/model.ckpt.data-00000-of-00001
ssdlite_mobilenet_v2_coco_2018_05_09/model.ckpt.meta
ssdlite_mobilenet_v2_coco_2018_05_09/model.ckpt.index
ssdlite_mobilenet_v2_coco_2018_05_09/saved_model/saved_model.pb


background tracking checkpoints

In [0]:
%cd /content/drive/My Drive/training_demo
%load_ext tensorboard
%tensorboard --logdir training/

In [0]:
%cd /content/drive/My Drive/training_demo
!python train.py --alsologtostderr --train_dir=training/ --pipeline_config_path=config_files/ssd_mobilenet_v1_coco.config


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
I0514 04:57:30.538679 139873514985344 learning.py:507] global step 13865: loss = 1.3726 (1.856 sec/step)
INFO:tensorflow:global step 13866: loss = 1.8946 (1.797 sec/step)
I0514 04:57:32.337545 139873514985344 learning.py:507] global step 13866: loss = 1.8946 (1.797 sec/step)
INFO:tensorflow:global step 13867: loss = 1.5789 (1.829 sec/step)
I0514 04:57:34.168213 139873514985344 learning.py:507] global step 13867: loss = 1.5789 (1.829 sec/step)
INFO:tensorflow:Recording summary at step 13868.
I0514 04:57:37.375587 139869967996672 supervisor.py:1050] Recording summary at step 13868.
INFO:tensorflow:global step 13868: loss = 1.5039 (3.220 sec/step)
I0514 04:57:37.389531 139873514985344 learning.py:507] global step 13868: loss = 1.5039 (3.220 sec/step)
INFO:tensorflow:global step 13869: loss = 1.4824 (1.798 sec/step)
I0514 04:57:39.189500 139873514985344 learning.py:507] global step 13869: loss = 1.4824 (1.798 sec

In [8]:
%cd /content/drive/My Drive/training_demo

!python models/research/object_detection/export_inference_graph.py --input_type image_tensor --pipeline_config_path config_files/ssd_mobilenet_v1_coco.config --trained_checkpoint_prefix training/model.ckpt-16250 --output_directory saved_trained_models/mobilenet_v1_2

/content/drive/My Drive/training_demo
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0514 08:30:10.677092 140386648606592 module_wrapper.py:139] From models/research/object_detection/export_inference_graph.py:145: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0514 08:30:10.684836 140386648606592 module_wrapper.py:139] From /content/drive/My Drive/training_demo/models/research/object_detection/exporter.py:402: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0514 08:30:10.685294 140386648606592 module_wrapper.py:139] From /content/drive/My Drive/training_demo/models/research/object_detection/exporter.py

KeyboardInterrupt: ignored